In [1]:
 # -*- coding:utf-8 -*-
import jieba
import pandas as pd
import numpy as np
import os
import xgboost
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
train_data_path = '/Users/yuanxihao/work/coder/python_workplace/kaggle/Ali-ATEC/atec_nlp.csv'
df_train = pd.read_csv(train_data_path)
print df_train.head


<bound method DataFrame.head of             id                                    s1  \
0            1                            怎么更改花呗手机号码   
1            2                       也开不了花呗，就这样了？完事了   
2            3                           花呗冻结以后还能开通吗   
3            4                              如何得知关闭借呗   
4            5                                花呗扫码付钱   
5            6                            花呗逾期后不能分期吗   
6            7                                花呗分期清空   
7            8                              借呗逾期短信通知   
8            9                      借呗即将到期要还的账单还能分期吗   
9           10                         花呗为什么不能支付手机交易   
10          11                       在吗，双***有临时花呗额度吗   
11          12                            我什么时候开通了花呗   
12          13                花呗每月还了最低还款后还会有万分之五的利息吗   
13          14                          我想用蚂蚁借呗怎么用不了   
14          15                               花呗也不能用了   
15          16                           花呗的安全没有验证成功   
16          17  

In [9]:
'''
中文停用词
'''
stopwords_path = "stopwords.txt"
stopwords = []
for word in open(stopwords_path,"r").readlines():
    stopwords.append(word.strip('\n'))

In [10]:
print stopwords[:10]

['!', '"', '#', '$', '%', '&', "'", '(', ')', '\t*']


In [11]:
'''
jieba中文分词
input: str
output: list of words
'''
def jieba_cut(sentence):
    seg_list = jieba.cut(sentence)
    return ",".join(seg_list).split(',')

import json
def json_dump(object_):
    return json.dumps(object_, ensure_ascii=False, encoding='utf8', indent=4)

In [12]:
'''
word match share
'''
def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in jieba_cut(row['s1']):
        if word not in stopwords:
            q1words[word] = 1
    for word in jieba_cut(row['s2']):
        if word not in stopwords:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q = [w for w in q1words.keys() if w in q2words]
    
    # 这里一定要注意 float，否则最后计算结果为 int
    R = float(len(shared_words_in_q) * 2)/(len(q1words) + len(q2words))
    return R

In [13]:
'''
tfidf word match share
'''
def tfidf_word_match_share(row):
    q1words = {}
    q2words = {}
    for word in jieba_cut(row['s1']):
        if word not in stopwords:
            q1words[word] = 1
    for word in jieba_cut(row['s2']):
        if word not in stopwords:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    
    shared_weights = [weights.get(w, 0) for w in q1words.keys() if w in q2words] + [weights.get(w, 0) for w in q2words.keys() if w in q1words]
    total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]
    
    R = np.sum(shared_weights) / np.sum(total_weights)
    return R

In [15]:
train_qs = pd.Series(df_train['s1'].tolist() + df_train['s2'].tolist()).astype(str)

In [17]:
print train_qs[:10]
print len(train_qs)

0          怎么更改花呗手机号码
1     也开不了花呗，就这样了？完事了
2         花呗冻结以后还能开通吗
3            如何得知关闭借呗
4              花呗扫码付钱
5          花呗逾期后不能分期吗
6              花呗分期清空
7            借呗逾期短信通知
8    借呗即将到期要还的账单还能分期吗
9       花呗为什么不能支付手机交易
dtype: object
204954


In [ ]:
train_word_match = df_train.apply(word_match_share, axis=1, raw=True)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/b0/cdwjddzs199007c6hhw6qgh80000gn/T/jieba.cache
Loading model cost 0.446 seconds.
Prefix dict has been built succesfully.
/Users/yuanxihao/.pyenv/versions/2.7.14/lib/python2.7/site-packages/ipykernel_launcher.py:8: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  
/Users/yuanxihao/.pyenv/versions/2.7.14/lib/python2.7/site-packages/ipykernel_launcher.py:11: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
plt.figure(figsize=[15,5])
plt.hist(train_word_match[df_train['is_duplicate'] == 0], bins=20, normed=True, label='not duplicate')
plt.hist(train_word_match[df_train['is_duplicate'] == 1], bins=20, normed=True, alpha=0.7, label='duplicate')
plt.legend()
plt.title('Label distribution over word_match_share', fontsize=15)
plt.xlabel('word_match_share', fontsize=15)
